In [3]:
import tensorflow as tf
from tensorflow import keras

In [3]:
# Load images from disk (data folder) and rescale them
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [4]:
# Preprocess all the images
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
# Implement a CNN to tackle the FER-2013 dataset
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 3, activation="relu", input_shape=[48, 48, 1]), # images are 28 × 28 pixels, with a single color (grayscale) 
    keras.layers.Conv2D(64, 3, activation="relu"),
    keras.layers.MaxPooling2D(2), # divide each spatial dimension by a factor of 2
    keras.layers.Dropout(0.25), 
    
    keras.layers.Conv2D(128, 3, activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Dropout(0.25),
    
    # Next is the fully connected network, composed of a hidden dense layer and a dense output layer.
    keras.layers.Flatten(), # convert each input image into a 1D array
    keras.layers.Dense(1024, activation="relu"),
    keras.layers.Dropout(0.5), # dropout rate of 50% to reduce overfitting
    keras.layers.Dense(7, activation="softmax"), # Dense output layer with 7 neurons (one per class) 
])

2021-11-15 19:28:32.597951: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-15 19:28:34.288589: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-15 19:28:34.288627: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist
2021-11-15 19:28:34.289403: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-15 19:28:34.430824: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 1995400000 Hz
2021-11-15 19:28:34.431937: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5a040013d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-15 19:28:

In [6]:
# Display all the model's layers 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         1

In [7]:
# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.Adam(lr=0.0001, decay=1e-6),
              metrics=["accuracy"])

In [7]:
# train the model 
history = model.fit(train_generator, steps_per_epoch=28709 // 64,
                    epochs=50, validation_data=validation_generator,
                    validation_steps=7178 // 64 )

Epoch 1/50
448/448 [==============================] - 122s 272ms/step - loss: 1.8007 - accuracy: 0.2625 - val_loss: 1.6969 - val_accuracy: 0.3527
Epoch 2/50
448/448 [==============================] - 128s 285ms/step - loss: 1.6198 - accuracy: 0.3683 - val_loss: 1.5265 - val_accuracy: 0.4205
Epoch 3/50
448/448 [==============================] - 127s 284ms/step - loss: 1.5104 - accuracy: 0.4197 - val_loss: 1.4403 - val_accuracy: 0.4482
Epoch 4/50
448/448 [==============================] - 127s 284ms/step - loss: 1.4370 - accuracy: 0.4516 - val_loss: 1.3748 - val_accuracy: 0.4754
Epoch 5/50
448/448 [==============================] - 125s 279ms/step - loss: 1.3738 - accuracy: 0.4763 - val_loss: 1.3216 - val_accuracy: 0.4933
Epoch 6/50
448/448 [==============================] - 127s 283ms/step - loss: 1.3231 - accuracy: 0.4987 - val_loss: 1.2953 - val_accuracy: 0.5093
Epoch 7/50
448/448 [==============================] - 128s 287ms/step - loss: 1.2755 - accuracy: 0.5178 - val_loss: 1.2471 -

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
    
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

NameError: name 'history' is not defined

In [9]:
# Evaluate the model 
model.evaluate(validation_generator)

113/113 [==============================] - 8s 68ms/step - loss: 1.9397 - accuracy: 0.2473


[1.939704179763794, 0.24728336930274963]

In [28]:
# Save the model’s architecture & the values of all the model parameters for every layer (since we're using the Sequential API)
model.save("emotion_model.h5")

In [29]:
                                ########################
                                # Face detection phase #
                                ########################

In [5]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image

In [6]:
# load the model 
model = keras.models.load_model("emotion_model.h5", compile=False)

2021-11-21 13:12:35.496519: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-21 13:12:36.830709: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-21 13:12:36.830763: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist
2021-11-21 13:12:36.831400: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-21 13:12:36.857239: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 1995460000 Hz
2021-11-21 13:12:36.858438: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe380000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-21 13:12:

In [ ]:
# Haar cascade based detector 
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# start the webcam feed
cap = cv2.VideoCapture(0)

while True:
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY) 

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7) 
        roi_gray=gray_img[y:y+w,x:x+h]  #cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255
        
        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        # Age prediction
        MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
        age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
        
        age_net = cv2.dnn.readNetFromCaffe('deploy_age.prototxt', 'age_net.caffemodel') 

        blob = cv2.dnn.blobFromImage(test_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        age_net.setInput(blob)
        age_preds = age_net.forward()
        predicted_age = age_list[age_preds[0].argmax()]
        
        label = "{} | {}".format(predicted_emotion, predicted_age)
        cv2.putText(test_img, label, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion/age analysis', resized_img)
    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [10]:
# Prepare the deployment to the RPI3
# https://www.tensorflow.org/lite/convert

# Convert the model 
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open('emotion_model_no_opt.tflite', 'wb') as f:
    f.write(tflite_model)

2021-11-20 11:19:30.725066: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-11-20 11:19:30.725194: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-11-20 11:19:30.727050: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:797] Optimization results for grappler item: graph_to_optimize
2021-11-20 11:19:30.727083: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:799]   function_optimizer: function_optimizer did nothing. time = 0.002ms.
2021-11-20 11:19:30.727088: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:799]   function_optimizer: function_optimizer did nothing. time = 0ms.
2021-11-20 11:19:30.853554: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-11-20 11:19:30.853706: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-11-20 11:19:30.898459: I tensorflo